In [1]:
!git clone https://github.com/nkt780426/Bachelor-s-Project.git

Cloning into 'Bachelor-s-Project'...
remote: Enumerating objects: 603, done.
remote: Counting objects: 100% (280/280), done.
remote: Compressing objects: 100% (213/213), done.
remote: Total 603 (delta 76), reused 265 (delta 62), pack-reused 323 (from 1)
Receiving objects: 100% (603/603), 129.13 MiB | 38.84 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [2]:
!mv Bachelor-s-Project/going_modular .

In [1]:
import os

import torch
from torchinfo import summary
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

from going_modular.dataloader.magface import create_magface_dataloader
from going_modular.model.MagFaceRecognition import MagFaceRecognition
from going_modular.train_eval.train import fit
from going_modular.loss.MagLoss import MagLoss
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
device

'cuda'

In [3]:
CONFIGURATION = {
    # Thư mục
    'type': 'albedo',
    'train_dir': '/kaggle/input/albedo/Dataset/Albedo/train',
    'test_dir': '/kaggle/input/albedo/Dataset/Albedo/test',
    
    # Cấu hình train
    'backbone': 'iresnet152',
    'epochs': 2000,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 224,
    'num_class': len(os.listdir('/kaggle/input/albedo/Dataset/Albedo/train')),
    'embedding_size': 512,
    
    'learning_rate': 0.1,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'alpha': 0.9,
    
    # Hàm m(ai) giúp thay đổi ai từ 0.25 đến 1.6
    'scale': 64,
    'lambda_g': 20,
    'l_margin': 0.45, 
    'u_margin': 0.8,
    'l_a': 10, 
    'u_a': 110,
}

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/albedo/Dataset/Albedo/train'

In [6]:
# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

In [7]:
train_dataloader, val_dataloader = create_magface_dataloader(CONFIGURATION, None, None)

In [8]:
model = MagFaceRecognition(CONFIGURATION).to(device)
criterion = MagLoss(conf = CONFIGURATION)
optimizer = torch.optim.Adam(model.parameters(), lr=CONFIGURATION['learning_rate'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=40, T_mult=1, eta_min=1e-6)
checkpoint_path = os.path.abspath('checkpoint/magface/albedo/' + CONFIGURATION['backbone'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)
earlystop_dir = os.path.abspath('checkpoint/magface/albedo/' + CONFIGURATION['backbone'] + '/models')
early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['val_euclidean_accuracy', 'val_cosine_accuracy', 'val_auc_euclidean', 'val_auc_cosine'],
    patience=40,
    mode='max',
    verbose=1,
    save_dir=earlystop_dir,
    start_from_epoch=60
)            

In [9]:
fit(CONFIGURATION, 0, model, device, train_dataloader, val_dataloader, criterion, optimizer, scheduler, early_stopping, modle_checkpoint)

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1:
	train: loss 3.890 | loss id   3.87 | top_1_acc 0.0052 | top_5_acc 0.0224 | acc_eu: 0.506 | acc_cos: 0.880 | auc_eu: 0.589 | auc_cos: 0.500
	val: acc_eu: 0.745 | acc_cos: 0.897 | auc_eu: 0.631 | auc_cos: 0.479
	Saving model to /kaggle/working/checkpoint/magface/albedo/iresnet152/models/checkpoint.pth


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 2:
	train: loss 3.831 | loss id   3.81 | top_1_acc 0.0060 | top_5_acc 0.0306 | acc_eu: 0.758 | acc_cos: 0.400 | auc_eu: 0.577 | auc_cos: 0.578
	val: acc_eu: 0.787 | acc_cos: 0.370 | auc_eu: 0.586 | auc_cos: 0.585
	Saving model to /kaggle/working/checkpoint/magface/albedo/iresnet152/models/checkpoint.pth


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

Epoch 34:
	train: loss 1.709 | loss id   1.69 | top_1_acc 0.0052 | top_5_acc 0.0284 | acc_eu: 0.628 | acc_cos: 0.234 | auc_eu: 0.567 | auc_cos: 0.519
	val: acc_eu: 0.631 | acc_cos: 0.261 | auc_eu: 0.583 | auc_cos: 0.550
	Saving model to /kaggle/working/checkpoint/magface/albedo/iresnet152/models/checkpoint.pth


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 35:
	train: loss 1.695 | loss id   1.67 | top_1_acc 0.0037 | top_5_acc 0.0277 | acc_eu: 0.587 | acc_cos: 0.121 | auc_eu: 0.575 | auc_cos: 0.515
	val: acc_eu: 0.644 | acc_cos: 0.271 | auc_eu: 0.596 | auc_cos: 0.524
	Saving model to /kaggle/working/checkpoint/magface/albedo/iresnet152/models/checkpoint.pth


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


KeyboardInterrupt: 

In [10]:
!zip -r /kaggle/working/checkpoint.zip /kaggle/working/checkpoint

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  adding: kaggle/working/checkpoint/ (stored 0%)
  adding: kaggle/working/checkpoint/magface/ (stored 0%)
  adding: kaggle/working/checkpoint/magface/albedo/ (stored 0%)
  adding: kaggle/working/checkpoint/magface/albedo/iresnet152/ (stored 0%)
  adding: kaggle/working/checkpoint/magface/albedo/iresnet152/models/ (stored 0%)
  adding: kaggle/working/checkpoint/magface/albedo/iresnet152/models/checkpoint.pth (deflated 12%)
  adding: kaggle/working/checkpoint/magface/albedo/iresnet152/logs/ (stored 0%)
  adding: kaggle/working/checkpoint/magface/albedo/iresnet152/logs/AUC_cosine_val/ (stored 0%)
  adding: kaggle/working/checkpoint/magface/albedo/iresnet152/logs/AUC_cosine_val/events.out.tfevents.1733996725.812a441182f3.23.10 (deflated 57%)
  adding: kaggle/working/checkpoint/magface/albedo/iresnet152/logs/AUC_euclidean_train/ (stored 0%)
  adding: kaggle/working/checkpoint/magface/albedo/iresnet152/logs/AUC_euclidean_train/events.out.tfevents.1733996725.812a441182f3.23.7 (deflated 59%)
 

In [5]:
%load_ext tensorboard
%tensorboard --logdir checkpoint/magface/albedo/iresnet152/logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 10003), started 0:00:05 ago. (Use '!kill 10003' to kill it.)